## Demonstração do serviço de texto para fala (Speech to text) Amazon Polly


Aqui você verá um resumo das funcionalidades que a Polly possui. 
O texto de entrada do serviço pode ser puro ou no formato SSML. O formato SSML é uma marcação de texto padrão W3C. Esse padrão serve para descrever a forma com que um dado texto deve ser 'lido' pelo sintetizador. Com os exemplos abaixo ficará mais claro como usar esse formato.

Obs.: Todo o conteúdo está em Português.

Então vamos lá!

In [ ]:
# Make sure you've installed boto3 version >=1.4.8
!conda upgrade --yes boto3

In [ ]:
import boto3
import numpy as np
import base64
import os
import time

from io import BytesIO
from IPython.core.display import display, HTML, Javascript, display_javascript

In [ ]:
err_message = "Try shutdown your kernel and start it again. It seems an old version was cached"
assert boto3.__version__ >= "1.4.8", err_message

Depois de importar as bibliotecas, vamos criar um cliente para a Polly e um mixer. O Mixer será utilizado para a reprodução do áudio retornado pela Polly.

In [ ]:
client = boto3.client('polly')


In [ ]:
html = """
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.2.1/jquery.min.js"></script>
<audio id="audio"><source id="media" src="audio.mp3" type="audio/mpeg" /></audio>
<script type="text/javascript">    
    function play_audio(){
        $("#audio")[0].load();
        $("#audio")[0].play();
    } // play_audio
</script>
"""
HTML(html)

Agora vamos definir algumas funções auxiliares para:
- tocar um streaming de áudio
- sintetizar a fala a partir de um arquivo SSML
- carregar um arquivo do disco

In [ ]:
def play_audio(audiostream):
    audio = open('audio.mp3', 'wb+')
    audio.write(audiostream.read())
    audio.flush()
    audio.close()
    
    jso = Javascript('play_audio()')
    display_javascript(jso)

In [ ]:
def sintetiza(ssml, lexicons=[], voiceid='Ricardo' ):
    if os.path.isfile('audio.mp3'):
        os.remove('audio.mp3') 
    response = client.synthesize_speech(
        LexiconNames=lexicons,
        OutputFormat='mp3',
        SampleRate='22050',
        Text=ssml,
        TextType='ssml',
        VoiceId=voiceid
    )
    play_audio( response['AudioStream'] )
    print(ssml)

In [ ]:
def load_file( filename ):
    return "".join( open( filename, 'r').readlines() )

Ok. Agora vamos fazer um breve teste de leitura. Note as tags &lt;speak&gt; e &lt;p&gt;. Elas fazem parte do padrão SSML.

In [ ]:
sintetiza( load_file( 'poema.ssml' ) )

### Tags SSML suportadas pela Polly
http://docs.aws.amazon.com/polly/latest/dg/supported-ssml.html
 - &lt;break&gt;: Intervalo em seg ou mili
 - &lt;lang&gt;: Corrige pronúncia de um estrangeirismo
 - &lt;mark&gt;: Adiciona marcações no texto, sem alterar a reprodução
 - &lt;p&gt;: Configura um novo parágrafo com uma pausa.
 - &lt;phoneme&gt;: Interpreta fonemas
 - &lt;prosody&gt;: Regula a cadência da fala: velocidade, entonação, etc
 - &lt;s&gt;: Dá uma pausa após ler a sentença
 - &lt;say-as&gt;: Muda a forma de se ler um texto: hora, telefone, etc
 - &lt;sub&gt;: Substitui um termo por uma sentença
 - &lt;w&gt;: Altera a pronúncia de uma palavra
 - &lt;amazon:effect name="whispered"&gt;: Sussurrar
 - &lt;amazon:effect name="drc"&gt;: Realçar som




### P & Lang

In [ ]:
sintetiza( load_file( 'lang.ssml' ) )

## Break

In [ ]:
sintetiza( load_file( 'break.ssml' ) )

## Say as

In [ ]:
sintetiza( load_file( 'sayas.ssml' ) )

## Lexicons

#### Carrega o arquivo de lexicons

In [ ]:
acronimos = load_file( 'lexicons.pls' )
print(acronimos)
response = client.put_lexicon(
    Name='acronimos',
    Content=acronimos
)

In [ ]:
sotaques = {
    'uai': 'uai.pls',
    'tche': 'tche.pls',
    'meu': 'meu.pls'
}
for nome, nome_arquivo in sotaques.items():
    response = client.put_lexicon(
        Name=nome,
        Content=load_file( nome_arquivo )
    )

In [ ]:
sintetiza( load_file( 'lexicon.ssml' ) )

In [ ]:
sintetiza( load_file( 'lexicon.ssml' ), [ 'acronimos'] )

In [ ]:
sintetiza( load_file( 'agua.ssml' ), [ 'acronimos'] )

#### Mineirês

In [ ]:
sintetiza( load_file( 'lexicon_sotaques.ssml' ), [ 'uai'] )

#### Paulistês

In [ ]:
sintetiza( load_file( 'lexicon_sotaques.ssml' ), [ 'meu'] )

#### Gauchês

In [ ]:
sintetiza( load_file( 'lexicon_sotaques.ssml' ), [ 'tche'] )

### Sussurrar & gritar

In [ ]:
sintetiza( load_file( 'alteracao_voz.ssml' ) )

### Breath

In [ ]:
ssml = """
<speak>
     Um ninho de mafagafos tem cinco mafagafinhos.<amazon:breath duration="short" volume="x-loud"/>
     Quem desmafagafizar os mafagafos bom desmafagafizador será.
</speak>
"""
sintetiza(ssml, voiceid='Ricardo')

In [ ]:
ssml = """
<speak>
   <amazon:auto-breaths>
Um Anel para a todos governar, Um Anel para encontrá-los, 
Um Anel para a todos trazer e na escuridão aprisioná-los. 
Na Terra de Mordor onde as Sombras se deitam." 
Por causa desse poema, muitas vezes os Anéis são referidos apenas como os Três, os Sete e os Nove, 
assim como o Anel de Sauron, chamado de o Um.
   </amazon:auto-breaths>
</speak>
"""
sintetiza(ssml, voiceid='Ricardo')

# Hack the following code and use the latest feature (Timbre)
http://docs.aws.amazon.com/polly/latest/dg/supported-ssml.html#vocaltractlength-tag


In [ ]:
# English voices: F(Joanna Salli Kimberly Kendra Ivy Matthew) M(Justin Joey)
# Spanish voices: F(Penelope) M(Miguel)
# Portuguese voices: F(Vitoria) M(Ricardo)
ssml = """
<speak>
 your text  <amazon:effect vocal-tract-length="+15%"> with the effect effect effect </amazon:effect> here
</speak>
"""
sintetiza(ssml, voiceid='Ivy')